In [2]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()


--2022-12-14 08:25:55--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.214.128, 173.194.215.128, 173.194.216.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2022-12-14 08:25:55 (141 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [3]:
class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get("accuracy") >= 0.999):
      print("Reached 99% training accuracy so stopping")
      self.model.stop_training = True

In [4]:
# data dir
train_dir = "/tmp/h-or-s"

In [5]:
# get data and load it
from keras.preprocessing import image

train_datagen = image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size = (150,150),
    class_mode = "binary"
)

Found 80 images belonging to 2 classes.


In [6]:
# create model

model = keras.models.Sequential([
    keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(150,150,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128, (3,3), activation="relu"),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(256, (3,3), activation="relu"),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 256)      0

In [7]:
# compile

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(lr=0.001), metrics=["accuracy"])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [8]:
callbacks = myCallback()
model.fit_generator(
    train_generator,
    epochs = 20,
    callbacks=[callbacks]
)

<ipython-input-8-59b0f6d5961a>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
3/3 [==============================] - 13s 4s/step - loss: 10.8502 - accuracy: 0.6500
Epoch 2/20
3/3 [==============================] - 10s 3s/step - loss: 0.7142 - accuracy: 0.4750
Epoch 3/20
3/3 [==============================] - 10s 4s/step - loss: 1.3073 - accuracy: 0.6875
Epoch 4/20
3/3 [==============================] - 10s 3s/step - loss: 0.7426 - accuracy: 0.4500
Epoch 5/20
3/3 [==============================] - 10s 4s/step - loss: 0.5373 - accuracy: 0.8500
Epoch 6/20
3/3 [==============================] - 10s 4s/step - loss: 0.5260 - accuracy: 0.7000
Epoch 7/20
3/3 [==============================] - 10s 3s/step - loss: 0.3441 - accuracy: 0.8500
Epoch 8/20
3/3 [==============================] - 10s 3s/step - loss: 0.1826 - accuracy: 0.9500
Epoch 9/20
3/3 [==============================] - 10s 4s/step - loss: 0.1604 - accuracy: 0.9750
Epoch 10/20
3/3 [==============================] - 10s 4s/step - loss: 0.1916 - accuracy: 0.9500
Epoch 11/20
3/3 [=====================

In [10]:
model.save("ex3.h5")